In [1]:
'''
# GPU BEGONE!
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu'
os.environ['PATH'] = '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ
'''

"\n# GPU BEGONE!\nimport os\nos.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu'\nos.environ['PATH'] = '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin'\nos.environ['CUDA_VISIBLE_DEVICES'] = '-1'\nos.environ\n"

In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd

# To find out which devices your operations and tensors are assigned to
tf.debugging.set_log_device_placement(True)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

# Obtain Embeddings

In [3]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased');

# model
model = TFBertModel.from_pretrained('bert-base-uncased');
model.summary()

Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device

# Import Data

In [4]:
dataset = pd.read_csv('dataset.csv', index_col=0)
#display(dataset.loc['Content'].values)

In [5]:
# grab content
dataset_content = dataset.loc['Content'].fillna('').values
#print(dataset_content)

# process content through BERT
encoded_strings = [tokenizer.encode(content, max_length=512, pad_to_max_length=True) for content in dataset_content]
encoded_vectors = tf.constant(encoded_strings)
outputs = model(encoded_vectors)

# obtain per line embeddings from all the artibles from BERT
last_hidden_states = outputs[0]

# obtain article embeddings from BERT
article_embeddings = outputs[1]

print(np.shape(outputs))
print(np.shape(outputs[0]))
print(np.shape(outputs[1]))

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
(2, 50)
(50, 512, 768)
(50, 768)


In [6]:
_, num_articles = np.shape(outputs)
article_names = [f'Article {idx}' for idx in range(num_articles)]

# vector embeddings of each article
vec_embeddings = article_embeddings.numpy()

In [8]:
#truth_labels = list(map(int, dataset.loc['Originality Score'].values))
#truth_labels = list(map(int, dataset.loc['Fact-Checked'].values))
truth_labels = list(map(int, dataset.loc['Credibility Score'].values))
print(f'Num truth_labels = {len(truth_labels)}')
print(truth_labels)
print(f'Unique Values: {set(truth_labels)} | Num Unique Values: {len(set(truth_labels))}')

Num truth_labels = 50
[3, 1, 4, 2, 1, 2, 0, 1, 0, 2, 1, 3, 0, 1, 1, 2, 0, 0, 2, 1, 4, 3, 0, 0, 1, 4, 4, 2, 4, 0, 1, 1, 2, 2, 4, 0, 4, 3, 4, 1, 3, 0, 1, 2, 1, 4, 4, 1, 1, 0]
Unique Values: {0, 1, 2, 3, 4} | Num Unique Values: 5


In [9]:
display(dataset)

,Article 1,Article 2,Article 3,Article 4,Article 5,Article 6,Article 7,Article 8,Article 9,Article 10,...,Article 41,Article 42,Article 43,Article 44,Article 45,Article 46,Article 47,Article 48,Article 49,Article 50
Title,"Floods in India, Bangladesh and Nepal kill 1,2...",NaN,Houston’s flooding shows what happens when you...,Psychologists believe Trump is showing signs o...,Vaccinated vs. Unvaccinated: Mawson Homeschool...,Dancing can reverse the signs of aging in the ...,Nations Press: PLS SHARE: DO NOT EAT THIS FISH...,"Preventing Alzheimer's Disease, Dementia & Cog...",Six Pharmaceutical Drugs That Immediately Dest...,"Under the GOP's health plan, sexual assault co...",...,"The oldest child is actually the smartest, stu...",DELINGPOLE: Global Warming Study Cancelled Bec...,Which Banana Would You Eat? Your Answer May Ha...,NaN,NaN,Opinion | How the Anti-Vaxxers Are Winning,Surgeon Gen.: Addiction Is A Chronic Brain Dis...,Study Finds Stevia Kills Lyme Disease Pathogen...,Diet drinks TRIPLE your risk of stroke and dem...,"The Amish, Who Don’t Get Vaccinated, Rarely Ge..."
Content,"At least 1,200 people have been killed and mil...",NaN,The city's gung-ho approach to development has...,Hillary Clinton claimed Trump was “temperament...,The Mawson study is a groundbreaking study of ...,As we grow older we suffer a decline in mental...,NaN,Preventing Alzheimer's naturally is as simple ...,Some pharmaceutical medicines can cause you im...,The bill would also mean insurers could consid...,...,"Birth order doesn't affect personality, but it...",A global warming research study in Canada has ...,"When we eat bananas, we do so because we are h...",NaN,NaN,A major measles outbreak in America is only a ...,The way forward includes needle exchanges and ...,Lyme disease is a very complicated disease to ...,"The Boston University study of almost 4,400 ad...",Do not talk about whether vaccines work or not...
URL,https://www.independent.co.uk/news/world/asia/...,https://www.ntd.tv/inspiring/life/9-sleeping-p...,https://qz.com/1064364/hurricane-harvey-housto...,https://www.independent.co.uk/life-style/healt...,https://info.cmsri.org/the-driven-researcher-b...,https://medicalxpress.com/news/2017-08-reverse...,http://www.nationspressph.com/2017/02/pls-shar...,https://dailyhealthpost.com/preventing-alzheim...,https://ewao.com/2017/09/16/six-pharmaceutical...,https://mic.com/articles/176092/under-the-gop-...,...,https://www.today.com/health/birth-order-first...,http://www.breitbart.com/big-government/2017/0...,http://goodfullness.net/which-banana-would-you...,https://www.ntd.tv/inspiring/parenting/sam-ber...,http://www.iflscience.com/health-and-medicine/...,https://www.nytimes.com/2017/02/08/opinion/how...,https://www.huffingtonpost.com/entry/vivek-mur...,https://www.collective-evolution.com/2017/01/2...,http://www.dailymail.co.uk/~/article-4429790/i...,https://worldtruth.tv/the-amish-who-dont-get-v...
Credibility Score,3,1,4,2,1,2,0,1,0,2,...,3,0,1,2,1,4,4,1,1,0
Originality Score,0,0,0,0,0,3,3,0,0,2,...,0,0,0,3,0,0,0,2,0,0
Attribution Rank,0,0,0,0,0,4,2,4,4,3,...,0,0,0,4,0,0,0,1,0,0
Fact-Checked,1,1,1,1,2,1,1,1,2,2,...,1,1,0,1,4,0,1,1,1,2
Source 1 Category,3,3,3,3,2,2,0,3,3,3,...,2,3,0,3,2,3,2,3,2,0
Source 1 Link Provided,2,2,2,2,2,2,0,2,2,2,...,2,2,0,2,2,1,2,1,1,2
Source 1 Impact Factor,could not locate,could not locate,could not locate,could not locate,3.991 (may be out of date),3.634,NaN,could not locate,6.99,could not locate,...,2.371,could not locate,NaN,could not locate,19.309,Could not locate.,could not locate,Could not locate.,Could not locate.,Could not locate.


# Define Neural Network Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Conv1D, MaxPooling1D, InputLayer, Input, Softmax

In [30]:
def model_def():
    model = Sequential([
        Input(shape=(768,), name='input'),
        Dense(128, activation='relu', name='dense_0'),
        Dense(64, activation='relu', name='dense_1'),
        #Dense(32, activation='relu', name='dense_2'),
        #Dense(len(set(truth_labels)), activation='softmax', name='dense_3'),

        #Dense(len(set(truth_labels)), name='dense_3'),

        #Dense(len(set(truth_labels)), activation='relu', name='dense_3'),
        #Dense(1000, name='dense_2'),
        #Softmax() # values too small, maybe try log-softmax

        Dense(len(set(truth_labels)), name='dense_3', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Softmax()
    ])
    model.summary()

    model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
                  # Loss function to minimize
                  loss=tf.keras.losses.SquaredHinge(),
                  #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  # List of metrics to monitor
                  metrics=['accuracy'])
    return model

In [22]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
kf.get_n_splits(vec_embeddings)
for train_index, test_index in kf.split(vec_embeddings):
    X_train = vec_embeddings[train_index]
    Y_train = truth_labels[test_index]
    
    X_test = vec_embeddings[test_index]
    Y_test = truth_labels[test_index]
    
    n_epochs = 1500
    history = model.fit(X_train, Y_train,
                        batch_size=10,
                        epochs=n_epochs, show=n_epochs%100==0);
                        # We pass some validation for
                        # monitoring validation loss and metrics
                        # at the end of each epoch
                        #validation_data=(x_val, y_val))
                    
    test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)
    #model.predict(X_test[0:1])

<generator object _BaseKFold.split at 0x7fbe982bfac0>

In [12]:
print(num_articles)
num_train_articles = 40
num_test_articles = num_articles - num_train_articles

50


In [13]:
input_vectors = vec_embeddings[:num_train_articles]
print(np.shape(input_vectors))

x_train = np.asarray(input_vectors)
print(x_train.shape)

output_vectors = np.array([truth_labels[i] for i in range(num_train_articles)])
print(np.shape(output_vectors))

y_train = output_vectors

(40, 768)
(40, 768)
(40,)


# Training

In [27]:
print('# Fit model on training data')
n_epochs = 1500
history = model.fit(x_train, y_train,
                    batch_size=10,
                    epochs=n_epochs, show=n_epochs%100==0);
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    #validation_data=(x_val, y_val))

print('\nhistory dict:', history.history)

# Fit model on training data
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.4970 - accuracy: 0.2500
Epoch 2/1500
4/4 [==============================] - 0

KeyboardInterrupt: 

In [15]:
x_test = vec_embeddings[num_train_articles:]
y_test = np.array([truth_labels[i] for i in range(num_train_articles, num_articles)])
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_test_function_26110 in device /job:localhost/replica:0/task:0/devic

In [16]:
model.predict(x_test[0:1])

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_26191 in device /job:localhost/replica:0/task:0/de

array([[0.19971475, 0.19976106, 0.19989948, 0.20031239, 0.20031233]],
      dtype=float32)